In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo

$$
\begin{split}
J_{\kappa \lambda} & = \sum_{i=1}^N  \sum_{\mu}^{M} \sum_{\nu}^{M} c_{i, \mu} c_{i, \nu} [\mu \nu | \kappa \lambda]\\
J_{\kappa \lambda} & = \sum_{i=1}^N  [\phi_i \phi_i | \kappa \lambda]
\end{split}
$$

In [2]:
def build_J(Fouridx,C,N):
    J = np.zeros(C.shape)
    for kappa in range(0,Fouridx.shape[0]):
        for lamda in range(0,Fouridx.shape[0]):
            for i in range(0,N): # all occupied
                for mu in range(0,Fouridx.shape[0]): # all basis functions
                    for nu in range(0,Fouridx.shape[0]): # all basis functions
                        J[kappa][lamda] += C[mu,i]*C[nu,i]*Fouridx[mu,nu,kappa,lamda]
    return 2*J # needs factor 2 because its spin restricted

$$
\begin{split}
K_{\kappa \lambda} & = \sum_{i=1}^N  \sum_{\mu}^{M} \sum_{\nu}^{M} c_{i, \mu} c_{i, \nu} [\mu  \kappa| \nu  \lambda]\\
K_{\kappa \lambda} & = \sum_{i=1}^N  [\phi_i \kappa | \phi_i \lambda]
\end{split}
$$

In [3]:
def build_K(Fouridx,C,N):
    K = np.zeros(C.shape)
    for kappa in range(0,Fouridx.shape[0]):
        for lamda in range(0,Fouridx.shape[0]):
            for i in range(0,N): # all occupied
                for mu in range(0,Fouridx.shape[0]): # all basis functions
                    for nu in range(0,Fouridx.shape[0]): # all basis functions
                        K[kappa][lamda] += C[mu,i]*C[nu,i]*Fouridx[mu,kappa,nu,lamda]
    return 2*K # needs factor 2 because its spin restricted

In [4]:
mol = gto.Mole()
mol.atom = """
    Ne    0.    0.    0.
"""
# this basis has 2 functions for Helium
mol.basis = "6-31g" #mol.basis = "ccpvdz", mol.basis = "sto-6g"
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 in the case of  case 16 distinct elements
eri = mol.intor('int2e')
S = mol.intor('int1e_ovlp')


In [5]:
mf = scf.RHF(mol)
mf.kernel()
J = mf.get_j()
K = mf.get_k()
C = mf.mo_coeff
N = mol.nelec[0]
P=np.matmul(C[:,0:N],C[:,0:N].T)

converged SCF energy = -128.473876870668



$$\bf{J}_{MO} = C^{T} \bf{J}_{AO} \bf{C}$$

and

$$\boldsymbol{\gamma} = \bf{C}^T \bf{S}^{T} \bf{P} \bf{S} \bf{C}$$

In [7]:
MO_J = np.matmul(C.T,np.matmul(J,C))
gamma =np.matmul(np.matmul(C.T,np.matmul(np.matmul(S,P),S)), C)


$$Tr[\boldsymbol{\gamma} \bf{J_{MO}}] = Tr[\bf{P} \bf{J}]$$

In [9]:
print(np.trace(np.matmul(MO_J,gamma)), np.trace(np.matmul(P,J)))

66.27654297875952
66.27654297875945
